In [3]:
import glob
import natsort
general_path = '/Users/emilio/Library/CloudStorage/Box-Box/GitHub/lightning-template/'
ob1_files = glob.glob(general_path+'./LabelDataObstetrics1/*.mat')
ob1_files = natsort.natsorted(ob1_files)
#split ob1_files.split('/')
patient_t=[]
sweep_t=[]
for i in range(len(ob1_files)):
    ob1_files[i]=ob1_files[i].split('/')[-1].split('.')[0]
    patient=ob1_files[i].split('_')[0]
    sweep=ob1_files[i].split('sweep')[1]
    #if sweep[0] is not a number, use sweep[1]
    sweep = sweep[1] if not sweep[0].isdigit() else sweep[0]
    patient_t.append(patient)
    sweep_t.append(sweep)
print(ob1_files)
print(patient_t)
print(sweep_t)

['001_sweep1', '001_sweep2', '001_sweep3', '001_sweep4', '001_sweep5', '002_sweep1', '002_sweep2', '002_sweep3', '002_sweep5', '002_sweep6', '003_sweep1', '003_sweep2', '003_sweep3', '003_sweep4', '004_sweep1', '004_sweep2', '004_sweep4', '004_sweep5', '004_sweep6', '004_sweep7', '004_sweep8', '005_sweep1', '005_sweep2', '005_sweep3', '005_sweep4', '005_sweep5', '005_sweep6', '006_sweep1', '006_sweep4', '006_sweep5', '006_sweep6', '006_sweep7', '007_sweep1', '007_sweep2', '007_sweep3', '007_sweep4', '007_sweep6', '007_sweep7', '007_sweep8', '008_sweep1', '008_sweep2', '008_sweep3', '008_sweep6', '008_sweep7', '008_sweep8', '009_sweep1', '009_sweep2', '009_sweep3', '009_sweep4', '009_sweep5', '009_sweep6', '009_sweep7', '010_sweep1', '010_sweep2', '010_sweep3', '010_sweep4', '010_sweep6', '011_sweep1 (1)', '011_sweep1', '011_sweep2 (1)', '011_sweep2', '011_sweep3 (1)', '011_sweep3', '011_sweep4 (1)', '011_sweep4', '011_sweep5 (1)', '011_sweep5', '011_sweep6 (1)', '011_sweep6', '012_swee

In [4]:


import numpy as np
import scipy.io as sio
import skvideo.io as skv
import torchio as tio
from skimage.transform import resize
import os
import sys
import matplotlib.pyplot as plt
import skimage.morphology as morph

src_path = os.path.join(general_path, 'src')
sys.path.append(src_path)

#import overlay_video from src/utils/overlay_video.py assuming that the file is in the same directory
from utils.overlay_video import generate_overlay_video

ob1_files = glob.glob(general_path+'./LabelDataObstetrics1/*.mat')
ob1_files = natsort.natsorted(ob1_files)

for i in range(0,99):
# for i in range(2,3):
     mp4_file_path = general_path+'./Study/'+patient_t[i]+'/'+patient_t[i]+'_sweep'+sweep_t[i]+'.mp4'
     ob = sio.loadmat(ob1_files[i])['labels']
     ob=np.moveaxis(ob, 2, 0)
     gt_video = skv.vread(mp4_file_path)
     diff = gt_video[1:,:,:,0]-gt_video[:-1,:,:,0]
     #remove small objects using morphological operations
     # diff = morph.remove_small_objects(diff.astype(bool), min_size=100)

     diff_sum = np.sum(diff,axis=(1,2))
     #normalize diff_sum
     diff_sum = diff_sum/np.max(diff_sum)

     # Assuming diff_sum is your normalized data array from the plot
     # Example: diff_sum = np.array([...])  # Replace with your actual array
     threshold = 0.2

     # Detect indices where diff_sum crosses above the threshold
     first_frame = np.where(diff_sum > threshold)[0][0]  # First upward crossing
     last_frame = np.where(diff_sum > threshold)[0][-1]  # Last upward crossing (before drop)
     # print(diff.shape)

     # Obtener los límites del área de interés
     row_limits, col_limits = np.where(morph.remove_small_objects(diff[first_frame+50].astype(bool), min_size=100) > 0)
     y1, y2 = row_limits.min(), row_limits.max()
     x1, x2 = col_limits.min(), col_limits.max()

     # Recortar el área de interés
     gt_video = gt_video[first_frame:last_frame, y1:y2, x1:x2, 0]
     ob = ob[first_frame:last_frame, :,:]
     #print ob frames where is not zero


     # Check shapes between gt_video and ob for width and height, and align them by cropping or padding centered
     if gt_video.shape[1] != ob.shape[1] or gt_video.shape[2] != ob.shape[2]:
          # Calculate padding or cropping for height (dim 1)
          pad_or_crop_h = (gt_video.shape[1] - ob.shape[1])
          pad_h_before = max(0, pad_or_crop_h // 2)
          pad_h_after = max(0, pad_or_crop_h - pad_h_before)
          crop_h_before = max(0, -pad_or_crop_h // 2)
          crop_h_after = max(0, -pad_or_crop_h - crop_h_before)

          # Calculate padding or cropping for width (dim 2)
          pad_or_crop_w = (gt_video.shape[2] - ob.shape[2])
          pad_w_before = max(0, pad_or_crop_w // 2)
          pad_w_after = max(0, pad_or_crop_w - pad_w_before)
          crop_w_before = max(0, -pad_or_crop_w // 2)
          crop_w_after = max(0, -pad_or_crop_w - crop_w_before)

          # Apply padding if needed
          if pad_h_before > 0 or pad_h_after > 0 or pad_w_before > 0 or pad_w_after > 0:
               ob = np.pad(ob, 
                              ((0, 0), 
                              (pad_h_before, pad_h_after), 
                              (pad_w_before, pad_w_after)), 
                              'constant')

          # Apply cropping if needed
          if crop_h_before > 0 or crop_h_after > 0 or crop_w_before > 0 or crop_w_after > 0:
               ob = ob[:, crop_h_before:ob.shape[1] - crop_h_after, crop_w_before:ob.shape[2] - crop_w_after]

     # Now, `ob` will have the same shape as `gt_video`
     # Create a TorchIO subject with paired data
     subject = tio.Subject(
     video_gt=tio.ScalarImage(tensor=np.expand_dims(gt_video,axis=0)),  # Add channel dimension
     label=tio.LabelMap(tensor=np.expand_dims(ob,axis=0))  # Add channel dimension
     )

     #efine the resize transform
     resize_transform = tio.transforms.Resize((256, 512,512))  # New shape: [frames, width, height]

     # Apply the resize transform to the subject
     resized_subject = resize_transform(subject)

     # Extract resized video GT and label from the subject
     resized_video_gt = resized_subject.video_gt.tensor.squeeze(0).numpy()  # Remove channel dimension
     resized_label = resized_subject.label.tensor.squeeze(0).numpy().astype(int)  # Ensure integers for label

     num_classes = 6  # Número de clases
     # Crear la versión one-hot encoded
     one_hot = np.eye(num_classes)[resized_label]  # Esto genera el one-hot encoding
     # Verificar el nuevo shape
     # print(one_hot.shape) 
     generate_overlay_video(resized_video_gt, one_hot, fps=30, output_video=general_path+'results/'+patient_t[i]+'_sweep'+sweep_t[i]+'_overlay.avi',gt_w=True)
     #save the label with ground truth in npz format
     np.savez_compressed(general_path+'results/'+patient_t[i]+'_sweep'+sweep_t[i]+'.npz', video=resized_video_gt, label=resized_label.astype(np.uint8))

     print(i)

Video guardado como /Users/emilio/Library/CloudStorage/Box-Box/GitHub/lightning-template/results/001_sweep1_overlay.avi
0
Video guardado como /Users/emilio/Library/CloudStorage/Box-Box/GitHub/lightning-template/results/001_sweep2_overlay.avi
1
Video guardado como /Users/emilio/Library/CloudStorage/Box-Box/GitHub/lightning-template/results/001_sweep3_overlay.avi
2
Video guardado como /Users/emilio/Library/CloudStorage/Box-Box/GitHub/lightning-template/results/001_sweep4_overlay.avi
3
Video guardado como /Users/emilio/Library/CloudStorage/Box-Box/GitHub/lightning-template/results/001_sweep5_overlay.avi
4
Video guardado como /Users/emilio/Library/CloudStorage/Box-Box/GitHub/lightning-template/results/002_sweep1_overlay.avi
5
Video guardado como /Users/emilio/Library/CloudStorage/Box-Box/GitHub/lightning-template/results/002_sweep2_overlay.avi
6
Video guardado como /Users/emilio/Library/CloudStorage/Box-Box/GitHub/lightning-template/results/002_sweep3_overlay.avi
7
Video guardado como /Use

In [5]:
# import matplotlib.pyplot as plt
# import numpy as np
# #for read mat file
# import scipy.io as sio
# import skvideo.io as skv
# import glob
# import torch
# general_path = '/Users/emilio/Documents/GitHub/lightning-template/'
# #compare mat files between LabelDataObstetrics and LabelDataObstetrics1
# mps_device = torch.device("mps")
# mp4_file_path = general_path+'Study/008/008_sweep7.mp4'


# ob_files = glob.glob(general_path+'/LabelDataObstetrics/*.mat')
# ob1_files = glob.glob(general_path+'./LabelDataObstetrics1/*.mat')
# mp4_path= glob.glob(general_path+'./Study/')


# maddie_002_1=sio.loadmat(general_path+'LabelDataObstetrics/008_sweep7 copy 2.mat')['labels']
# #move 3rd dimension to 1st dimension
# maddie_002_1 = np.moveaxis(maddie_002_1,2,0)
# marika_002_1=sio.loadmat(general_path+'LabelDataObstetrics1/008_sweep7.mat')['labels']
# #move 3rd dimension to 1st dimension
# marika_002_1 = np.moveaxis(marika_002_1,2,0)
# gt_video = skv.vread(mp4_file_path)
# print(maddie_002_1.shape)
# print(marika_002_1.shape)
# print(gt_video.shape)

# #check i where masks are not empty in maddie_002_1
# for i in range(maddie_002_1.shape[0]):
#     if np.sum(maddie_002_1[i])>0:
#         print(i)
#         break
# #check i where masks are not empty in marika_002_1
# for i in range(marika_002_1.shape[0]):
#     if np.sum(marika_002_1[i])>0:
#         print(i)
#         break



